In [ ]:
import pandas as pd

df = pd.read_parquet('selectie-energiematrix.parquet')
df.head()

In [ ]:
df.info(verbose=True, show_counts=True)

In [ ]:
df['INHARMSOC'].isnull().sum()

In [ ]:
# Fill in missing categorical values with the mode from that column

df['INHARMSOC'].fillna(value=df['INHARMSOC'].mode()[0], inplace=True)
df['INHARMSOC'].isnull().sum()

In [ ]:
# Fill in missing numerical value with mean from that column

df['INHAHL'].fillna(value=df['INHAHL'].mean(), inplace=True)
df['INHAHL'].isnull().sum()

In [ ]:
df.info(verbose=True, show_counts=True)

In [ ]:
df['label_dik'].groupby(df['bc2022']).agg(lambda x:x.value_counts().index[0])

In [ ]:
# Fill in missing energylabel with mode by buurtcode

df['label_dik'].fillna(value=df['label_dik'].groupby(df['bc2022']).transform(lambda x:x.value_counts().index[0]), inplace=True)
df['label_dik'].isnull().sum()

In [ ]:
df.drop(['Zonneboiler', 'Zonnepanelen'], axis=1, inplace=True)
df.info(verbose=True, show_counts=True)

In [ ]:
df.to_parquet('energiematrix_zonder_missing.parquet')

EDA for specific Gemert-Bakel data

In [ ]:
# Create new dataframe for only Gemert-Bakel

df_gb = df.loc[df['gem2022'] == 'Gemert-Bakel']
df_gb.info()

In [ ]:
# See descriptives for all numerical values

df_gb.describe()

In [ ]:
# The column VBOOPPERVLAKTE seems to have an outlier. Check this by creating plots

df_gb.hist(column='VBOOPPERVLAKTE', bins=15)

In [ ]:
df_gb.plot.scatter(x='VBOOPPERVLAKTE', y='lihk_lilek_nieuw')

In [ ]:
# Remove one outlier by filtering for everything in this column under 30000

df_gb = df_gb[df_gb['VBOOPPERVLAKTE'] < 30000]

In [ ]:
df_gb.plot.scatter(x='VBOOPPERVLAKTE', y='lihk_lilek_nieuw')

In [ ]:
df_gb.hist(column = 'VBOOPPERVLAKTE', bins=15)

In [ ]:
# Look closer at categorical variables by creating a seperate dataframe including only cateogirical variables

df_cat = df_gb.select_dtypes(include = 'category').copy()
df_cat.head(10)

In [ ]:
# Create seperate dataframe for value labels, number of missing values, percentages missing, etc. for all categorical variables

df_cat_stats = pd.DataFrame(columns = ['column', 'values', 'values_count_incna', 'num_miss', 'pct_miss'])
tmp = pd.DataFrame()

for c in df_cat.columns:
    tmp['column'] = [c]
    tmp['values'] = [df_cat[c].unique()]
    tmp['values_count_incna'] = len(list(df_cat[c].unique()))
    tmp['num_miss'] = df_cat[c].isnull().sum()
    tmp['pct_miss'] = (df_cat[c].isnull().sum()/len(df_cat)).round(3)*100
    df_cat_stats = df_cat_stats.append(tmp)
    
df_cat_stats

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data = df_gb, x = 'label_dik')

In [ ]:
# Visualise the y variable; lihk_lilek_nieuw

sns.countplot(data = df_gb, x = 'lihk_lilek_nieuw')

In [ ]:
# the graph above shows that the missing values have been replaced with -1
# Drop missing values from the y variable

df_gb = df_gb[df_gb['lihk_lilek_nieuw'] != -1]
df_gb.info('lihk_lilek_nieuw')

In [ ]:
# Plot the y variable again to check

sns.countplot(data = df_gb, x = 'lihk_lilek_nieuw')

In [ ]:
# Create correlationtable to check for correlations between numerical values

corrmat = df_gb.corr()
f, ax = plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=.9, square=True, annot=True)

In [ ]:
# Zoom in on correlations with the y variable

correlation_ea = df_gb.corr().loc['lihk_lilek_nieuw']
correlation_ea.sort_values(ascending = False)

In [ ]:
# Create multiple countplots to check division in categorical variables

sns.countplot(data = df_gb, x = 'VBOWoningtype', hue = 'lihk_lilek_nieuw')
plt.xticks(rotation=45)

In [ ]:
sns.countplot(data = df_gb, x = 'bouwjaarklasse_pbl', hue = 'lihk_lilek_nieuw')
plt.xticks(rotation=45)

In [ ]:
sns.countplot(data = df_gb, x = 'oppervlakteklasse_pbl', hue = 'lihk_lilek_nieuw')
plt.xticks(rotation=45)

In [ ]:
sns.countplot(data = df_gb, x = 'label_dik', hue = 'lihk_lilek_nieuw')
plt.xticks(rotation=45)